In [2]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
data = pd.read_csv("2009_2021Basketball.csv")


#Semi Ojeleye stats which I found online
data.loc[data['ast/tov'].isna(), 'ast/tov'] = 1.06

data['twoPM'] = data['twoPM'].astype('int64')
data['TPM'] = data['TPM'].astype('int64')

PER = []
netRtg = []
for i in range(len(data)):
    PER.append((((data.loc[i, 'twoPM'] + data.loc[i, 'TPM']) * 85.910)
               +((data.loc[i, 'GP'] * data.loc[i, 'stl']) * 53.897)
               +(data.loc[i, 'TPM'] * 51.757)
               +(data.loc[i, 'FTM'] * 46.845)
               +((data.loc[i, 'GP'] * data.loc[i, 'blk']) * 39.190)
               +((data.loc[i, 'GP'] * data.loc[i, 'oreb']) * 39.190)
               +((data.loc[i, 'GP'] * data.loc[i, 'ast']) * 34.677)
               +((data.loc[i, 'GP'] * data.loc[i, 'dreb']) * 14.707)
                -((data.loc[i, 'GP'] * data.loc[i, 'pfr']) * 17.174)
                -((data.loc[i, 'FTA'] - data.loc[i, 'FTM']) * 20.091)
                -(((data.loc[i, 'twoPA'] + data.loc[i, 'TPA']) - (data.loc[i, 'twoPM'] + data.loc[i, 'TPM'])) * 39.190)
                -(((data.loc[i, 'ast']) / (data.loc[i, 'ast/tov']) * (data.loc[i, 'GP'])) * 53.897))
                * (1 / (data.loc[i, 'mp'] * data.loc[i, 'GP'])))
    netRtg.append(data.loc[i, 'Ortg'] - data.loc[i, 'drtg'])
    
              
# https://bleacherreport.com/articles/113144-cracking-the-code-how-to-calculate-hollingers-per-without-all-the-mess            
data['PER'] = PER
data['netRtg'] = netRtg
data.head(10)



C:\Users\liams\AppData\Local\Temp\ipykernel_31200\4221356394.py:1: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("2009_2021Basketball.csv")
C:\Users\liams\AppData\Local\Temp\ipykernel_31200\4221356394.py:24: RuntimeWarning: invalid value encountered in double_scalars
  -(((data.loc[i, 'ast']) / (data.loc[i, 'ast/tov']) * (data.loc[i, 'GP'])) * 53.897))
C:\Users\liams\AppData\Local\Temp\ipykernel_31200\4221356394.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  * (1 / (data.loc[i, 'mp'] * data.loc[i, 'GP'])))


,player_name,team,conf,GP,Min_per,Ortg,usg,eFG,TS_per,ORB_per,...,dreb,treb,ast,stl,blk,pts,Unnamed: 64,Unnamed: 65,PER,netRtg
0,DeAndrae Ross,South Alabama,SB,26,29.5,97.3,16.6,42.5,44.43,1.6,...,0.6154,0.8077,1.1923,0.3462,0.0385,3.8846,NaN,6.22026,4.603199,-11.0210
1,Pooh Williams,Utah St.,WAC,34,60.9,108.3,14.9,52.4,54.48,3.8,...,1.2647,1.9412,1.8235,0.4118,0.2353,5.9412,NaN,3.94375,7.120085,3.2220
2,Jesus Verdejo,South Florida,BE,27,72.0,96.2,21.8,45.7,47.98,2.1,...,2.3333,2.9630,1.9630,0.4815,0.0000,12.1852,NaN,10.92680,8.264053,-11.3560
3,Mike Hornbuckle,Pepperdine,WCC,30,44.5,97.7,16.0,53.6,53.69,4.1,...,1.4333,2.1333,1.1000,0.5667,0.1333,4.9333,NaN,6.77427,8.035690,-11.1580
4,Anthony Brown,Pacific,BW,33,56.2,96.5,22.0,52.8,54.31,8.3,...,3.3030,4.7273,0.8485,0.4545,0.3333,7.5758,NaN,0.00000,9.470778,-4.5520
5,Nick Rodgers,Butler,Horz,6,0.7,0.0,0.0,0.0,0.00,0.0,...,0.3333,0.3333,0.3333,0.0000,0.0000,0.0000,NaN,0.00000,-50.701973,-92.7329
6,Dana Smith,Longwood,ind,27,77.8,104.8,23.0,53.4,56.30,6.8,...,3.8889,6.0370,1.8148,1.7778,0.7407,15.2593,NaN,3.22107,17.610147,3.5910
7,Matt Beck,Fordham,A10,19,10.4,131.9,3.3,90.0,90.00,2.6,...,0.1579,0.3158,0.1053,0.1053,0.0000,0.4737,NaN,2.50384,0.930489,13.3960
8,Justin Drummond,Wagner,NEC,30,82.8,99.7,20.5,48.8,53.07,2.9,...,3.9333,4.7333,4.1333,1.7333,0.8000,10.5000,NaN,3.42016,13.165177,2.1018
9,Jamal Smith,Wagner,NEC,30,80.4,92.5,23.0,43.5,45.29,6.4,...,3.4667,5.2000,1.7333,1.2333,0.1000,11.8667,NaN,3.34007,10.445487,-8.5070


In [16]:

data = data.drop(columns = ['Unnamed: 64', 'Unnamed: 65', 'rimmade', 'rimmade+rimmiss', 'midmade', 'midmade+midmiss'])
data = data.drop(columns = ['num', 'Rec Rank', 'rimmade/(rimmade+rimmiss)', 'midmade/(midmade+midmiss)', 'dunksmade', 'dunksmiss+dunksmade', 'dunksmade/(dunksmade+dunksmiss)'])
data = data.drop(columns = ['type', 'pid', 'adjoe', 'porpag', 'ht', 'Min_per', 'ORB_per', 'DRB_per', 'AST_per', 'TO_per', 'adrtg', 'dporpag', 'stops', 'gbpm', 'ogbpm', 'dgbpm'])
data = data.convert_dtypes()
data.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61061 entries, 0 to 61060
Data columns (total 39 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   player_name  61061 non-null  string 
 1   team         61061 non-null  string 
 2   conf         61061 non-null  string 
 3   GP           61061 non-null  Int64  
 4   Ortg         61061 non-null  Float64
 5   usg          61061 non-null  Float64
 6   eFG          61061 non-null  Float64
 7   TS_per       61061 non-null  Float64
 8   FTM          61061 non-null  Int64  
 9   FTA          61061 non-null  Int64  
 10  FT_per       61061 non-null  Float64
 11  twoPM        61061 non-null  Int64  
 12  twoPA        61061 non-null  Int64  
 13  twoP_per     61061 non-null  Float64
 14  TPM          61061 non-null  Int64  
 15  TPA          61061 non-null  Int64  
 16  TP_per       61061 non-null  Float64
 17  blk_per      61061 non-null  Float64
 18  stl_per      61061 non-null  Float64
 19  ftr 

In [5]:
print(data.isnull().sum())
data.isnull().sum(axis=1).sort_values(ascending = False) 

player_name        0
team               0
conf               0
GP                 0
Ortg               0
usg                0
eFG                0
TS_per             0
FTM                0
FTA                0
FT_per             0
twoPM              0
twoPA              0
twoP_per           0
TPM                0
TPA                0
TP_per             0
blk_per            0
stl_per            0
ftr                0
yr               274
pfr                0
year               0
ast/tov            0
pick           59626
drtg              45
bpm               45
obpm              45
dbpm              45
mp                38
oreb              38
dreb              38
treb              38
ast               38
stl               38
blk               38
pts               38
PER             3595
dtype: int64


8932     15
5260     15
6212     15
8923     15
27723    15
         ..
14004     0
5817      0
14006     0
18706     0
3682      0
Length: 61061, dtype: int64

In [6]:
data.loc[data['pick'].isna(), 'pick'] = 65
data['pick'] = data['pick'].astype('int64')

data.dropna(inplace = True)

data = data[data['pick'] < 61]



data = data.sort_values(['year', 'pick'], ascending = [True, True])



In [7]:
X   = data[['pts', 'GP', 'oreb', 'dreb', 'ast', 'ast/tov', 'mp', 'blk', 'stl', 'twoPM', 'TPM', 'twoPA', 'TPA', 'FTA', 'FTM', "pfr"]] #get the input features
y   = data['PER']              #get the target

X_train, X_test, y_train, y_test = train_test_split(X,              #the input features
                                                    y,              #the label
                                                    test_size=0.3,  #set aside 30% of the data as the test set
                                                    random_state=7 #reproduce the results
                                                   )

In [8]:
rf = RandomForestRegressor(random_state=7)
rf.fit(X_train, y_train)

RandomForestRegressor(random_state=7)

In [9]:
#y_pred = rf.predict(X_test)
#print('The predicted PER is: {}'.format(y_pred))

In [10]:
mse = mean_squared_error(y_test, y_pred)

# Evaluate the Predictions
print('The mse of the model is: {}'.format(mse))

NameError: name 'y_pred' is not defined

In [ ]:
data2 = pd.read_csv("draft-data-20-years.csv")

In [ ]:
data2

In [ ]:
#adding a new column BPM to data which represents the BPM of the drafted player in the NBA
data["BPM"] = [None]*len(data)
#for each player in the first dataset we try to find them in the second and assign that bmp to the first
#we can't always find the player so we print the ones we can't find for reference 
#we will remove them from our data 
for name in data['player_name']:
    try:
        data.loc[data['player_name'] == name,'BPM'] = data2.loc[data2['Player'] == name]['BPM'][data2[data2['Player'] == name].index[0]]
    except:
        print("couldn't find",name)
        
data.loc[data['player_name'] == 'Nikola Vucevic', 'BPM'] = 2.2
data.loc[data['player_name'] == 'Cameron Thomas', 'BPM'] = -3.6
data.loc[data['player_name'].str.contains('Hyland'), 'BPM'] = -0.2
data.loc[data['player_name'] == 'Xavier Tillman', 'BPM'] = 0.0	
data.loc[data['player_name'] == 'Nicolas Claxton', 'BPM'] = 0.7
data.loc[data['player_name'] == 'R.J. Barrett', 'BPM'] = -2.3
data.loc[data['player_name'] == 'PJ Washington', 'BPM'] = -1.0
data.loc[data['player_name'] == 'Robert Woodard', 'BPM'] = -7.7
data.loc[data['player_name'] == 'Jacob Evans III', 'BPM'] = -7.4
data.loc[data['player_name'] == 'Kezie Okpala', 'BPM'] = -4.7
data.loc[data['player_name'] == 'Temetrius Morant', 'BPM'] = 2.3
data.loc[data['player_name'] == 'Mohamed Bamba', 'BPM'] = 0.1
data.loc[data['player_name'] == 'Vincent Edwards', 'BPM'] = -6.4
data.loc[data['player_name'] == 'Dewan Huell', 'BPM'] = -9.6
data.loc[data['player_name'] == 'Bruce Brown Jr.', 'BPM'] = -0.9
data.loc[data['player_name'] == 'Sviatoslav Mykhailiuk', 'BPM'] = -2.7
data.loc[data['player_name'] == 'Wesley Iwundu', 'BPM'] = -3.9
data.loc[data['player_name'] == 'Vincent Edwards', 'BPM'] = -6.4
data.loc[data['player_name'] == 'Vincent Edwards', 'BPM'] = -6.4
data.loc[data['player_name'] == 'Vincent Edwards', 'BPM'] = -6.4
data.loc[data['player_name'] == 'Vincent Edwards', 'BPM'] = -6.4
data.loc[data['player_name'] == 'Vincent Edwards', 'BPM'] = -6.4
data.loc[data['player_name'] == 'Vincent Edwards', 'BPM'] = -6.4
data.loc[data['player_name'] == 'Vincent Edwards', 'BPM'] = -6.4
data.loc[data['player_name'] == 'Vincent Edwards', 'BPM'] = -6.4
data.loc[data['player_name'] == 'Vincent Edwards', 'BPM'] = -6.4
data.loc[data['player_name'] == 'Vincent Edwards', 'BPM'] = -6.4

In [ ]:
data

In [ ]:
data.info()

In [ ]:
#dropping all the players which we didn't have all the data for 
#data = data.dropna()
#data.info()

In [ ]:
data[["team",'player_name',"BPM"]].groupby('team').mean().sort_values(by="BPM",ascending=False).head(10)

In [ ]:
data[["team",'player_name',"BPM"]].groupby('team').size().sort_values(ascending=False).head(10)

In [ ]:
data[data['player_name'] == 'Jordan Nwora'][['player_name','year', 'pick']]

In [ ]:
#removing duplicate players by keeping highest year 
#sorting rows by year
data = data.sort_values(by=['year'], ascending=False)
#removing duplicates by keeping only first year 
data = data.drop_duplicates(subset=['player_name', 'pick'], keep='first')

In [ ]:
data[data['player_name'] == 'Jordan Nwora'][['player_name','year', 'pick']]

In [ ]:
data.info()

In [24]:
hm = sns.heatmap(data[data['pts', 'ast']])
  
# displaying the plotted heatmap
plt.show()

KeyError: ('pts', 'ast')